<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de pacotes

In [1]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


#Importar pacotes


In [23]:
import pandas as pd
import string
import time
import csv
from ast import literal_eval
from textblob import TextBlob
from google.colab import drive


#Baixar dataset

In [3]:
#baixa o arquivo do dataset para o ambiente colab
!wget "http://tiagodemelo.info/datasets/dataset-v2.dat"



--2020-11-11 00:17:48--  http://tiagodemelo.info/datasets/dataset-v2.dat
Resolving tiagodemelo.info (tiagodemelo.info)... 108.167.188.189
Connecting to tiagodemelo.info (tiagodemelo.info)|108.167.188.189|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89773865 (86M)
Saving to: ‘dataset-v2.dat’

dataset-v2.dat      100%[===================>]  85.61M  20.3MB/s    in 4.7s    

2020-11-11 00:17:54 (18.1 MB/s) - ‘dataset-v2.dat’ saved [89773865/89773865]



In [4]:
# ler o arquivo com dos dados do dataset baixado
file = open("./dataset-v2.dat",'r')

# sepera em linhas os dados
rows = file.readlines()

# fecha o arquivo
file.close()

# criar uma lista para salvar os dicionarios
dictionaries = []

for row in rows:
  #transforma literal em dicionario
  saveDic = literal_eval(row)

  #adiciona atributos de subdicionarios no dicionario mais externo
  saveDic.update(saveDic['itemReviewed'].items())
  saveDic.update(saveDic['itemReviewed']['address'].items())

  # #delete subdicionarios
  del saveDic['address']
  del saveDic['itemReviewed']

  #adiciona na lista de dicionarios
  dictionaries.append(saveDic)




In [5]:
# transforma a lista de dicionario em dataframe
data = pd.DataFrame(dictionaries)

data.head()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,name,website,email,phoneNumber,averageRating,streetAddress,addressLocality,addressRegion,postalCode,country
0,Jantar,692856288,2344932,larinhavercosa222,"Comida maravilhosa, lugar do verdadeiro chicle...",pt,True,"São Carlos, SP",26/07/2019,07/2019,5,Imperador dos Camarões,http://www.imperadordoscamaroes.com.br,contato@imperadordoscamaroes.com.br,+55 82 3231-4134,4.5,Avenida Doutor Antônio Gouveia,,Alagoas,57030-170,Brasil
1,Uma boa experiência.,229208487,3720906,Pano P,"Comida boa, drinks legais e um ambiente bem ac...",pt,True,"Manaus, AM",16/09/2014,None,4,Guacamole Natal,http://www.facebook.com/GuacamoleNatal,guacamolenatal@gmail.com,+55 84 3219-3646,4.5,Avenida Praia de Ponta Negra 9068 Ponta Negra,,Rio Grande do Norte,59092-100,Brasil
2,Otimo self service,679002332,2248105,Magma100,"Excelente pra ir com a família! Ótima Comida, ...",pt,True,"Goiânia, GO",04/06/2019,07/2018,4,Chica Pitanga,http://www.chicapitanga.com.br,chicapitanga@chicapitanga.com.br,+55 81 3334-8260,4.5,Rua Petrolina 19,Recife,Pernambuco,51021-250,Brasil
3,O sorvete mais tradicional do estado,355671519,8168302,Yurie M,Tomo sorvete na Visabor faz mais de 30 anos. S...,pt,False,"Aracaju, SE",15/03/2016,01/2016,5,Sorvetes Vi Sabor,None,None,None,4.0,Avenida Francisco Porto 59 Grageru,Aracaju,Sergipe,,Brasil
4,Melhor hamburger!,268741921,5395141,Debora_C85,O melhor hamburger que já comi! Excelente aten...,pt,False,"Rio de Janeiro, RJ",28/04/2015,04/2015,5,Z Deli Sandwich Shop,http://www.facebook.com/zdelisandwichshop/,None,551130830021,4.5,Rua Haddock Lobo 1386 Jardins,São Paulo,Estado de São Paulo,01414-002,Brasil


#Tratamento de Comentários Vazios

In [6]:
#seleciona tuplas vazias
data[pd.isnull(data.reviewBody)]

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,name,website,email,phoneNumber,averageRating,streetAddress,addressLocality,addressRegion,postalCode,country
8528,Vale a pena conhecer!,224445539,NaN,NaN,NaN,pt,False,"Recife, PE",25/08/2014,12/2013,4,Outback Steakhouse - Shopping RioMar Recife,https://www.outback.com.br/restaurantes/shoppi...,faleconosco@outback.com.br,+55 81 3035-0930,4.5,"Av República do Líbano, 256 Shopping RioMar Re...",Recife,Pernambuco,51110-160,Brasil


In [7]:
data.reviewBody.fillna('VAZIO')

data.reviewBody[8].encode('utf-16', 'surrogatepass').decode('utf-16')

'Ambiente agradável, preço totalmente acessível, e adoro o cardápio de lá! Sem falar no atendimento maravilhoso! Voltarei sempre 😘'

#Obter sentenças

**Text Blob**

In [66]:
# criar uma lista para salvar os blobs
comentario_blob = []

#Obter lista dos comentários
comentarios = data.reviewBody

# Iteração em cada review
for comentario in comentarios:
  #Obter sentenças tratando emojis
  aux = TextBlob(str(comentario).encode('utf-16', 'surrogatepass').decode('utf-16'))
  for sentence in aux.sentences:
    if sentence != "!" and sentence != "." and sentence != ".." and sentence != "rs" and sentence != "Aff." :
      comentario_blob.append(sentence)

In [67]:
#Imprima as 10 primeiras sentenças
comentario_blob[:10]

[Sentence("Comida maravilhosa, lugar do verdadeiro chiclete de camarão."),
 Sentence("O garçom Éric Lima nos atendeu muito bem, pretendemos voltar assim que possível."),
 Sentence("Comida boa, drinks legais e um ambiente bem aconchegante."),
 Sentence("Provamos muitas coisas e tudo nos agradou."),
 Sentence("Principalmente um pato servido em um dos pratos da casa, sensacional!"),
 Sentence("Excelente pra ir com a família!"),
 Sentence("Ótima Comida, ambiente agradável, bom atendimento e estacionamento."),
 Sentence("Vale a pena conhecer"),
 Sentence("Tomo sorvete na Visabor faz mais de 30 anos."),
 Sentence("Sabores de frutas ou artificiais.")]

#Salvar no Drive

Montar drive

In [68]:
drive.mount('/content/drive/', force_remount='true')

Mounted at /content/drive/


Salvar no drive documento com comentários para tradução posterior. 

In [104]:
!ls -la 'drive/My Drive/sentiment_analysis_project/'

drive_file = "drive/My Drive/sentiment_analysis_project/translate.csv" 
file = open(drive_file,'r')

data_translate = pd.read_csv(file)

# Iteração em cada review
for cmt in comentarios:
  try:
    #Obter sentenças tratando emojis
    comment_aux = str(cmt).encode('utf-16', 'surrogatepass').decode('utf-16')
    #cria objeto para adicionar ao dataframe
    obj_aux = { 'review': comment_aux, 'review_us': 'EMPTY' }
    # adiciona ao dataframe
    data_translate = data_translate.append(obj_aux, ignore_index=True)
  except:
    print("exception: " + cmt)

# Salva arquivo no google Drive
data_translate.to_csv(drive_file, index=False, header=True)    
data_translate

total 1
-rw------- 1 root root 18 Nov 11 01:16 translate.csv


,review,review_us
0,"Comida maravilhosa, lugar do verdadeiro chicle...",EMPTY
1,"Comida boa, drinks legais e um ambiente bem ac...",EMPTY
2,"Excelente pra ir com a família! Ótima Comida, ...",EMPTY
3,Tomo sorvete na Visabor faz mais de 30 anos. S...,EMPTY
4,O melhor hamburger que já comi! Excelente aten...,EMPTY
...,...,...
99995,"Ainda não inaugurado oficialmente, está atende...",EMPTY
99996,"Bom atendimento, lugar bonito, comida muito go...",EMPTY
99997,O Rio Scenarium é uma das baladas mais famosas...,EMPTY
99998,"Fui por indicação e com boa expectativa!\n116,...",EMPTY


#Traduzir sentenças

Comentários traduzidos para Inglês (US) a partir de CSV no google drive.

In [133]:
drive_file = "drive/My Drive/sentiment_analysis_project/translate_us.csv" 

data_translate = pd.read_csv(drive_file)

dt = data_translate.loc[data_translate['review_us'] == 'EMPTY']

#Obter lista dos comentários
comentarios = dt.review

# Iteração em cada review. Colocar o número de interações que deseja realizar.
for comentario in comentarios[:5]:
  try:
    time.sleep(1)
    # Obter sentenças tratando emojis
    txt_blob_aux = TextBlob(str(comentario).encode('utf-16', 'surrogatepass').decode('utf-16'))
    aux_us = str(txt_blob_aux.translate(to='us'))
    print(aux_us)

    # Atualiza dataframe
    data_translate.at[(data_translate['review'] == comentario), 'review_us'] = aux_us
  except:
      print("EXCEPTION: " + str(txt_blob_aux))


# Salva arquivo no google Drive
data_translate.to_csv(drive_file, index=False, header=True)

data_translate

I have had ice cream at Visabor for over 30 years. Fruit or artificial flavors. Crunchy cone and simple atmosphere. Very good prices. I recommend
The best hamburger I've ever eaten! Excellent service, the food was great. The restaurant is quite small, but nice and fun.
The restaurant is new, super cozy and charming with great service. The portions are generous, very tasty and at a fair price. Good selection of wines. I highly recommend the seafood spaghetti, bolognese, carbonara and risotto. The tiramisu is also fantastic.
Every time I went with my boyfriend, but it is an excellent environment to take family or friends.
We went on a Saturday, elegant atmosphere, kind and friendly staff. We ate a very tasty curry, but the filet was not filet. The dessert (pineapple wonton with pepper jelly and ice cream) was great.
Great food with good attendance and with a lot of good things to take your wife and girlfriend and family because the food portions are good!
thank you


,review,review_us
0,"Comida maravilhosa, lugar do verdadeiro chicle...","Wonderful food, the place for real shrimp gum...."
1,"Comida boa, drinks legais e um ambiente bem ac...","Good food, nice drinks and a very cozy atmosph..."
2,"Excelente pra ir com a família! Ótima Comida, ...","Excellent to go with the family! Great food, p..."
3,Tomo sorvete na Visabor faz mais de 30 anos. S...,I have had ice cream at Visabor for over 30 ye...
4,O melhor hamburger que já comi! Excelente aten...,The best hamburger I've ever eaten! Excellent ...
...,...,...
99995,"Ainda não inaugurado oficialmente, está atende...",EMPTY
99996,"Bom atendimento, lugar bonito, comida muito go...",EMPTY
99997,O Rio Scenarium é uma das baladas mais famosas...,EMPTY
99998,"Fui por indicação e com boa expectativa!\n116,...",EMPTY
